In [ ]:
]st

In [ ]:
using Revise

In [ ]:
using DataFrames, Query, TaylorSeries, PlanetaryEphemeris

In [ ]:
using JLD

In [ ]:
using NEOs

In [ ]:
using Statistics

In [ ]:
using LinearAlgebra

In [ ]:
using Dates

In [ ]:
using Gaston

In [ ]:
using StatsBase

In [ ]:
using DelimitedFiles

In [ ]:
NEOs.loadjpleph()

In [ ]:
## 16 ast + radial nongrav (A1 / SRP)
#dq = set_variables("δx", order=5, numvars=8)
#radec_bwd = JLD.load("radec_99942_2004_2020.dat.jld", "radec_table")
#radec_fwd = JLD.load("radec_99942_2020_2021.dat.jld", "radec_table")
#deldop_bwd = JLD.load("deldop_99942_RADAR_2005_2013.dat.jld", "deldop_table")
#deldop_fwd_ = JLD.load("deldop_99942_RADAR_2021.dat.jld", "deldop_table")

# 32 ast (NO srp)
dq = set_variables("δx", order=5, numvars=7)
radec_bwd = JLD.load("radec_99942_2004_2020_32AST.dat.jld", "radec_table")
radec_fwd = JLD.load("radec_99942_2020_2021_32AST.dat.jld", "radec_table")
deldop_bwd = JLD.load("deldop_99942_RADAR_2005_2013_32AST.dat.jld", "deldop_table")
deldop_fwd_ = JLD.load("deldop_99942_RADAR_2021_32AST.dat.jld", "deldop_table")

## 16 ast (NO srp)
#dq = set_variables("δx", order=5, numvars=7)
#radec_bwd = JLD.load("radec_99942_2004_2020_LAST_16ast.dat.jld", "radec_table")
#radec_fwd = JLD.load("radec_99942_2020_2021_LAST_16ast.dat.jld", "radec_table")
#deldop_bwd = JLD.load("deldop_99942_RADAR_2005_2013_LAST_16ast.dat.jld", "deldop_table")
#deldop_fwd_ = JLD.load("deldop_99942_RADAR_2021_LAST_16ast.dat.jld", "deldop_table")
;

In [ ]:
radec_all_ = vcat(radec_bwd, radec_fwd)
radec_all = radec_all_ |> @filter( 
    (Date(_.dt_utc_obs) != Date(2021, 1, 28)) #&& (Date(_.dt_utc_obs) <= Date(2021, 3, 8))
    ) |> DataFrame
;

In [ ]:
#deldop_fwd = deldop_fwd_[1:end-1,:] # in paper results, only 3 out of 4 radar measurements are used
deldop_fwd = deldop_fwd_

deldop_all = vcat(deldop_bwd, deldop_fwd)
;

In [ ]:
### first and last dates (optical obs)
radec_all.dt_utc_obs[1], radec_all.dt_utc_obs[end]

In [ ]:
nrow(radec_all), nrow(deldop_all)

In [ ]:
del_bwd = deldop_bwd |> @filter(_.delay_index) |> DataFrame
dop_bwd = deldop_bwd |> @filter(_.doppler_index) |> DataFrame

del_fwd = deldop_fwd |> @filter(_.delay_index) |> DataFrame
dop_fwd = deldop_fwd |> @filter(_.doppler_index) |> DataFrame

del_all = deldop_all |> @filter(_.delay_index) |> DataFrame
dop_all = deldop_all |> @filter(_.doppler_index) |> DataFrame
;

In [ ]:
res_τ = del_all |> @map( _.τ_obs - _.τ_comp ) |> collect
res_ν = dop_all |> @map( _.ν_obs - _.ν_comp ) |> collect

σ_del_bwd = del_bwd.σ_τ
σ_dop_bwd = dop_bwd.σ_ν

σ_del_fwd = del_fwd.σ_τ
σ_dop_fwd = dop_fwd.σ_ν

σ_del_all = del_all.σ_τ
σ_dop_all = dop_all.σ_ν

w_τ = del_all |> @map( 1/(_.σ_τ)^2 ) |> collect
w_ν = dop_all |> @map( 1/(_.σ_ν)^2 ) |> collect


# only Vokrouhlicky et al. (2015) data
res_τ_v15 = del_bwd |> @map( _.τ_obs - _.τ_comp ) |> collect
res_ν_v15 = dop_bwd |> @map( _.ν_obs - _.ν_comp ) |> collect

w_τ_v15 = del_bwd |> @map( 1/(_.σ_τ)^2 ) |> collect
w_ν_v15 = dop_bwd |> @map( 1/(_.σ_ν)^2 ) |> collect
;

In [ ]:
# check if all RA/Dec observations are CCD:

all(radec_all |> @map(begin x = _.j2000; x == "C" || x == "X" end))

In [ ]:
# there are 494 RA/Dec obs without catalog info
nrow(  radec_all |> @filter(_.catalog == " ") |> DataFrame  )

In [ ]:
### WIP: outlier rejection

### function chisqobs(row, Γ_β, x, outlier::Bool=false)
###     A_α_TN = TaylorSeries.gradient(row.α_obs-row.α_corr-row.α_comp)
###     A_δ_TN = TaylorSeries.gradient(row.δ_obs-row.δ_corr-row.δ_comp)
###     A_i = hcat(A_α_TN(x), A_δ_TN(x))
###     outlier_sign = outlier*2-1
###     (A_i')*Γ_β*A_i
### end

### chisqobs(res0, Γ_OR7, x_OR7)

In [ ]:
#radec_04XX = radec_0415
#radec_04XX = radec_0420
#radec_04XX = radec_0421
#radec_04XX = filter(x->( DateTime(x.yr, x.month, x.day) != DateTime(2021, 1, 28) ), radec_0421)
#radec_04XX = radec_0421

#radec_04XX = filter(x->x.dt_utc_obs<DateTime(2021,1,14), radec_0420)


#radec_04XX = filter(x->((x.α_obs-x.α_corr-x.α_comp())^2 + (x.δ_obs-x.δ_corr-x.δ_comp())^2)<χ2_rej, radec_04XX)
#radec_04XX = filter(x->((x.α_obs-x.α_corr-x.α_comp(x_OR7))^2 + (x.δ_obs-x.δ_corr-x.δ_comp(x_OR7))^2)/(x.σ^2) ≤ χ2_rej, radec_04XX)

In [ ]:
#χ2_rej = 4.0 #4.0 #4.25 #5.025 # Brozovic et al (2018) use Chesley et al (2010) outlier rejection, in turn from Carpino et al (2003)
radec_04XX = radec_all
#radec_04XX = filter(x->((x.α_obs-x.α_corr-x.α_comp(x_OR7))^2 + (x.δ_obs-x.δ_corr-x.δ_comp(x_OR7))^2)/(x.σ^2) ≤ χ2_rej, radec_04XX)
#;

In [ ]:
#nrow(radec_all), nrow(radec_04XX)

In [ ]:
#radec_04XX.dt_utc_obs

In [ ]:
# Vokrouhlicky et al. (2015) obs table
radec_vok15 = radec_table("../data/vokr15_mpc_formatted.dat")

# vector of RA values from Vokrouhlicky et al. (2015) observations (used for filtering)
vok15_rav = radec_vok15.α_obs

vok15_ind_filt = findall(x->x ∈ vok15_rav, radec_all.α_obs)
;

In [ ]:
# Vorkouhliky et al. (2015) RA/Dec table
radec_v15 = radec_all |> @filter(_.α_obs ∈ vok15_rav) |> DataFrame;

In [ ]:
radec_v15_radec_corr = radec_vok15 |> @select( :α_corr, :δ_corr ) |> DataFrame;

In [ ]:
res_α = radec_04XX |> @map(_.α_obs - _.α_corr - _.α_comp) |> collect
res_δ = radec_04XX |> @map(_.δ_obs - _.δ_corr - _.δ_comp) |> collect

res_α_v15 = (radec_v15 |> @map(_.α_obs - _.α_comp)) .- radec_v15_radec_corr.α_corr
res_δ_v15 = (radec_v15 |> @map(_.δ_obs - _.δ_comp)) .- radec_v15_radec_corr.δ_corr
;

In [ ]:
mean(res_α()), mean(res_δ())

In [ ]:
# read astrometric errors from Tholen et al. (2013)
tho13_errors = readdlm("../data/tholenetal2013_opterror.dat", ',')
# compute weights
w_α_tho13 = 1 ./ (tho13_errors[:,1].^2 .+ tho13_errors[:,3].^2 .+ tho13_errors[:,5].^2)
w_δ_tho13 = 1 ./ (tho13_errors[:,2].^2 .+ tho13_errors[:,4].^2 .+ tho13_errors[:,6].^2)

# Tholen et al. (2013) obs table
radec_tho13 = radec_table("../data/tholen13_mpc_formatted.dat")
# vector of RA values from Tholen et al. (2013) observations (used for filtering)
tho13_rav = radec_tho13.α_obs

tho13_ind_filt = findall(x->x ∈ tho13_rav, radec_all.α_obs)
tho13_ind_filt_v15 = findall(x->x ∈ tho13_rav, radec_v15.α_obs)
@show length(tho13_ind_filt), length(tho13_ind_filt_v15)
;

In [ ]:
function w8sfarnocchia15(row::NamedTuple)
    return w8sfarnocchia15(row.obscode, row.dt_utc_obs, row.catalog)
end

function w8sfarnocchia15(df::DataFrame)
    return w8sfarnocchia15.(df.obscode, df.dt_utc_obs, df.catalog)
end

function w8sfarnocchia15(obscode::Union{Int,String}, dt_utc_obs::DateTime, catalog::String)
    w = 1.0 # unit weight (arcseconds)
    if obscode == "H01" # Magdalena Ridge
        return 0.3w
    elseif obscode == "568" # Mauna Kea (MPEC 2014-R71)
        return 0.13w #0.2w #0.2":Verevs17 weight;0.13":Farnocchia15 weight
    elseif obscode == "F51" # PanSTARRS-PS1
        return 0.15w #0.2w #0.2":Verevs17 weight;0.15":Farnocchia15 weight
    else
        return w
    end
end


In [ ]:
# for each observation batch, count the number of observations made in the same night (?) by the same observatory
date_obscode_v = radec_04XX |> @map(  (Date(_.dt_utc_obs),_.obscode)  )
date_obscode_v_v15 = radec_v15 |> @map(  (Date(_.dt_utc_obs),_.obscode)  )
Nv = [count(x->x[1]==i[1] && x[2]==i[2], date_obscode_v) for i in date_obscode_v]
Nv_v15 = [count(x->x[1]==i[1] && x[2]==i[2], date_obscode_v_v15) for i in date_obscode_v_v15]
relax_factor = map(x->x>4.0 ? x/4.0 : 1.0, Nv)
relax_factor_v15 = map(x->x>4.0 ? x/5.0 : 1.0, Nv_v15) # map(x->x>2.0 ? x : 1.0, Nv_v15)

# statistical weights according to Veres et al (2017)
σ_v17 = radec_04XX.σ
σ_v17_v15 = w8sfarnocchia15(radec_v15)

w_α = 1 ./ (σ_v17.^2)
w_δ = 1 ./ (σ_v17.^2)

w_α_v15 = 1 ./ (σ_v17_v15.^2)
w_δ_v15 = 1 ./ (σ_v17_v15.^2)

# Tholen et al. (2013) astrometry uncertainties
w_α[tho13_ind_filt] .= w_α_tho13
w_δ[tho13_ind_filt] .= w_δ_tho13
w_α_v15[tho13_ind_filt_v15] .= w_α_tho13
w_δ_v15[tho13_ind_filt_v15] .= w_δ_tho13

w_α = w_α./(relax_factor)
w_δ = w_δ./(relax_factor)

w_α_v15 = w_α_v15./(relax_factor_v15)
w_δ_v15 = w_δ_v15./(relax_factor_v15)
;

In [ ]:
length(w_τ), length(w_ν), length(w_α), length(w_δ)

In [ ]:
length(w_τ_v15), length(w_ν_v15), length(w_α_v15), length(w_δ_v15)

In [ ]:
nobs_R = nrow(del_all) + nrow(dop_all)
nobs_O = nrow(radec_04XX)
nobs = nobs_R + nobs_O
@show nobs_O, nobs_R, nobs
;

In [ ]:
res_O = vcat(res_α, res_δ)
w_O = vcat(w_α, w_δ)

res_R = vcat(res_τ, res_ν)
w_R = vcat(w_τ, w_ν)

res_OR = vcat(res_R, res_O)
w_OR = vcat(w_R, w_O)
;

In [ ]:
# only Vokrouhliky et al. (2015) data
res_O_v15 = vcat(res_α_v15, res_δ_v15)
w_O_v15 = vcat(w_α_v15, w_δ_v15)

res_R_v15 = vcat(res_τ_v15, res_ν_v15)
w_R_v15 = vcat(w_τ_v15, w_ν_v15)

res_OR_v15 = vcat(res_R_v15, res_O_v15)
w_OR_v15 = vcat(w_R_v15, w_O_v15)
;

In [ ]:
# pre-2021, with SRP ("OR8")

x_OR7_v15, Γ_OR7_v15 = newtonls(res_OR_v15, w_OR_v15, zeros(get_numvars()))
σ_OR7_v15 = sqrt.(diag(Γ_OR7_v15))
@show x_OR7_v15 σ_OR7_v15 x_OR7_v15[end]/σ_OR7_v15[end]

x_OR6_v15, Γ_OR6_v15 = newtonls_6v(res_OR_v15, w_OR_v15, zeros(get_numvars()))
σ_OR6_v15 = sqrt.(diag(Γ_OR6_v15))
@show x_OR6_v15 σ_OR6_v15 x_OR6_v15[end]/σ_OR6_v15[end]

In [ ]:
@show x_OR7_v15;
@show Γ_OR7_v15;

In [ ]:
# post-2021, NO SRP, 32 ast

x_OR7, Γ_OR7 = newtonls(res_OR, w_OR, zeros(get_numvars()))
σ_OR7 = sqrt.(diag(Γ_OR7))
@show x_OR7 σ_OR7 x_OR7[end]/σ_OR7[end]

x_OR6, Γ_OR6 = newtonls_6v(res_OR, w_OR, zeros(get_numvars()))
σ_OR6 = sqrt.(diag(Γ_OR6))
@show x_OR6 σ_OR6 x_OR6[end]/σ_OR6[end]
;

In [ ]:
(-2.898835108503474 - -2.8988788886209065), (-2.898835108503474 - -2.8988788886209065)/0.02474582241296382

In [ ]:
# post-2021, NO SRP, 16 ast

x_OR7, Γ_OR7 = newtonls(res_OR, w_OR, zeros(get_numvars()))
σ_OR7 = sqrt.(diag(Γ_OR7))
@show x_OR7 σ_OR7 x_OR7[end]/σ_OR7[end]

x_OR6, Γ_OR6 = newtonls_6v(res_OR, w_OR, zeros(get_numvars()))
σ_OR6 = sqrt.(diag(Γ_OR6))
@show x_OR6 σ_OR6 x_OR6[end]/σ_OR6[end]
;

In [ ]:
@show x_OR7 Γ_OR7

In [ ]:
#x_O7, Γ_O7 = newtonls(res_O, w_O, zeros(8))
#σ_O7 = sqrt.(diag(Γ_O7))
#@show x_O7 σ_O7 x_O7[end]/σ_O7[end]
#
#x_R7, Γ_R7 = newtonls(res_R, w_R, zeros(8))
#σ_R7 = sqrt.(diag(Γ_R7))
#@show x_R7 σ_R7 x_R7[end]/σ_R7[end]
#
#x_O6, Γ_O6 = newtonls(res_O, w_O, zeros(8))
#σ_O6 = sqrt.(diag(Γ_O6))
#@show x_O6 σ_O6 x_O6[end]/σ_O6[end]
#
#x_R6, Γ_R6 = newtonls(res_R, w_R, zeros(8))
#σ_R6 = sqrt.(diag(Γ_R6))
#@show x_R6 σ_R6 x_R6[end]/σ_R6[end]

In [ ]:
#@show x_OR7 Γ_OR7

In [ ]:
# 32 ast
#sqrt(Γ_OR7[7,7])

In [ ]:
# 16 ast
sqrt(Γ_OR7[7,7])

In [ ]:
abs(-2.894053174513909 +2.8941071491385224)/sqrt(Γ_OR7[7,7])

In [ ]:
#@show x_OR7 Γ_OR7

In [ ]:
@show x_OR7.*vcat(1e-8ones(6), 1e-14)
;

In [ ]:
#@show x_OR7.*vcat(1e-8ones(6), 1e-14, 1e-13)
#;

# Fit quality statistics: $\chi^2$, normalized RMS

The gravity-only solution post-fit normalized RMS for the optical, radar, and combined optical and radar astrometry are, respectively, 0.448, 14.720 and 1.0259. Moreover, the gravity-only solution post-fit right-ascension weighted mean residual is 0.072$~$arcsec, while the declination weighted mean residual is 0.218$~$arcsec. The weighted time-delay and Doppler shift mean residual for the gravity-only solution are, respectively, -0.06 us and 0.043 Hz. On the other hand, the non-gravitational solution normalized RMS for the optical, radar, and combined optical and radar astrometry are, respectively, 0.313, 0.387 and 0.314. The right-ascension, declination, time-delay and Doppler shift weighted mean residuals, are, respectively, 0.008$~$arcsec, 0.003$~$arcsec, -0.0003$~$us and -0.31$~$Hz.

In [ ]:
mean(res_α(x_OR6), weights(w_α)), mean(res_δ(x_OR6), weights(w_δ))

In [ ]:
mean(res_α(x_OR7), weights(w_α)), mean(res_δ(x_OR7), weights(w_δ))

In [ ]:
mean(res_τ(x_OR6), weights(w_τ)), mean(res_ν(x_OR6), weights(w_ν))

In [ ]:
mean(res_τ(x_OR7), weights(w_τ)), mean(res_ν(x_OR7), weights(w_ν))

In [ ]:
nrms(res_O(x_OR6), w_O), nrms(res_R(x_OR6), w_R), nrms(res_OR(x_OR6), w_OR)

In [ ]:
nrms(res_O(x_OR7), w_O), nrms(res_R(x_OR7), w_R), nrms(res_OR(x_OR7), w_OR)

In [ ]:
chi2(res_OR(x_OR7), w_OR)

In [ ]:
chi2(res_OR(x_OR6), w_OR)

In [ ]:
# signal-to-noise ratio
abs(x_OR7[end]/σ_OR7[end])

# Semimajor axis drift

In [ ]:
jd0 = datetime2julian(DateTime(2020,12,17)) #Julian date (TDB) of integration initial time
et0 = (jd0 - J2000)*daysec

In [ ]:
q00 = [-0.18034747703273316, 0.9406910666200128, 0.3457360259054398, -0.016265942170279046, 4.392889725556651e-5, -0.00039519931615139716]

In [ ]:
if get_numvars() == 7
    @show q00 .+ (vcat(1e-8*ones(6), 1e-14).*x_OR7)[1:6]
elseif get_numvars() == 8
    q00 .+ (vcat(1e-8*ones(6), 1e-14, 1e-13).*x_OR7)[1:6]
end

In [ ]:
xas = q00 .- NEOs.kmsec2auday(NEOs.sun_pv(et0)) # .+ (vcat(1e-8*ones(6), 1e-14).*x_OR7)[1:6]

In [ ]:
ϵ0_deg = 84381.448/3600;
m_eq2ecl = Rx(deg2rad(ϵ0_deg));
m_xv_eq2ecl = hcat(vcat(m_eq2ecl, zeros(3,3)), vcat(zeros(3,3), m_eq2ecl));

In [ ]:
kep0 = vcat([pv2kep(  m_xv_eq2ecl*(xas .+ (vcat(1e-8*ones(6), 1e-14).*dq)[1:6])  , μ[1], jd0)...][1:6],1e-14(dq[7])); # x_OR6 +
#kep0 = vcat([pv2kep(  m_xv_eq2ecl*(xas .+ (vcat(1e-8*ones(6), 1e-14, 1e-13).*dq)[1:6])  , μ[1], jd0)...][1:6],1e-14(dq[7]),1e-13(dq[8])); # x_OR6 +
kep0()

In [ ]:
kep_OR7 = kep0(x_OR7)

In [ ]:
# WARNING: these are ecliptic elements!!! also, not sure about the order in the Ω ↔ ω angles
kep_JPL216 = [0.1914347568286823, 0.7460634895090179, 2459424.558511739019, 203.9662692226386, 126.5900238507279, 3.338874353538208, -2.901085583204654E-14, 5.E-13]
#σ_kep_JPL204 = [2.2197e-09, 2.2003e-09, 1.2886e-06, 1.4045e-05, 1.4205e-05, 2.4295e-07, 3.039E-16]
;

In [ ]:
kep_JPL216

In [ ]:
orbel_OR7_TN = kep_OR7 .+ dq
#orbel_OR7_TN = kep_JPL216 .+ dq # length(kep_JPL216) == 8

In [ ]:
var_names_str = prod(get_variable_names().*" ")
vars_old = get_variables()
vars_new = set_variables(eltype(vars_old[1]), var_names_str[1:prevind(var_names_str, end)], order=vars_old[1].order)

In [ ]:
x_OR7[7], σ_OR7[7]

In [ ]:
# OR7, 16 ast
ady_kep_au_My = 1e6yr*yarkp2adot(orbel_OR7_TN[7], orbel_OR7_TN[2]/(1.0 - orbel_OR7_TN[1]), orbel_OR7_TN[1], μ[1]) # au/Myr

In [ ]:
# OR7, 16 ast
ady_kep_m_y = 1e3au*yr*yarkp2adot(orbel_OR7_TN[7], orbel_OR7_TN[2]/(1.0 - orbel_OR7_TN[1]), orbel_OR7_TN[1], μ[1]) #m/yr

In [ ]:
- 198.70516356300044 - (- 198.70886945315476)

In [ ]:
t_car2kep(x) = TaylorSeries.jacobian(kep0, x)'
t_kep2ady(x, ady_kep) = TaylorSeries.gradient(ady_kep)(x)

In [ ]:
Γ_OR7_kep = (t_car2kep(x_OR7)')*Γ_OR7*(t_car2kep(x_OR7))

In [ ]:
1e4sqrt( (t_kep2ady(kep_OR7, ady_kep_au_My)')*Γ_OR7_kep*t_kep2ady(kep_OR7, ady_kep_au_My) )

In [ ]:
sqrt( (t_kep2ady(kep_OR7, ady_kep_m_y)')*Γ_OR7_kep*t_kep2ady(kep_OR7, ady_kep_m_y) )

In [ ]:
sqrt( (t_kep2ady(kep_OR7, ady_kep_m_y)')*Γ_OR7_kep*t_kep2ady(kep_OR7, ady_kep_m_y) )

In [ ]:
t_car2kep(x_OR7)

In [ ]:
σ_OR7_kep = sqrt.(diag(Γ_OR7_kep))

In [ ]:
kep_OR7

In [ ]:
kep_JPL216

In [ ]:
σ_OR7_kep

In [ ]:
Γ_OR7_kep = (t_car2kep(x_OR7)')*Γ_OR7*(t_car2kep(x_OR7));
Γ_OR6_kep = (t_car2kep(x_OR6)[1:6,1:6]')*Γ_OR6*t_car2kep(x_OR6)[1:6,1:6];

In [ ]:
jd0

In [ ]:
for (x, Γ_kep, sol_str) in [(x_OR6, Γ_OR6_kep, "OR6"), (x_OR7, Γ_OR7_kep, "OR7")]
    println("* ----===OOO===--- *", sol_str)
    @show kep0(x) sqrt.(diag(Γ_kep))
end

In [ ]:
for (x, Γ, sol_str) in [(x_OR6, Γ_OR6, "OR6"), (x_OR7, Γ_OR7, "OR7")]
    println("* ----===OOO===--- *", sol_str)
    σ_sol_ = sqrt.(diag(Γ))
    σ_sol = length(σ_sol_)==6 ? 1e-8σ_sol_[1:6] : vcat(1e-8σ_sol_[1:6], 1e-14σ_sol_[7])
    @show vcat(q00, 0).+vcat(1e-8x[1:6], 1e-14x[7]) σ_sol
end

In [ ]:
res_α_OR6 = res_α(x_OR6)
res_δ_OR6 = res_δ(x_OR6)

res_α_OR7 = res_α(x_OR7)
res_δ_OR7 = res_δ(x_OR7)
;

In [ ]:
mean(res_α_OR6), std(res_α_OR6), mean(res_δ_OR6), std(res_δ_OR6)

In [ ]:
mean(res_α_OR7), std(res_α_OR7), mean(res_δ_OR7), std(res_δ_OR7)

# Plots: optical astrometry post-fit residuals

In [ ]:
methods(fit)

In [ ]:
#bins_hist = -2.0:0.1:2.0
nbins_plot = 100

#h_a = fit(Histogram, res_α_OR6, bins_hist)
h_a = fit(Histogram, res_α_OR6, nbins=nbins_plot)
bc_a = midpoints(h_a.edges[1]) # bin centers
bs_a = fill(h_a.edges[1].step.hi, length(bc_a)) # bin sizes
;

#h_b = fit(Histogram, res_δ_OR6, bins_hist)
h_b = fit(Histogram, res_δ_OR6, nbins=nbins_plot)
bc_b = midpoints(h_b.edges[1]) # bin centers
bs_b = fill(h_b.edges[1].step.hi, length(bc_b)) # bin sizes

#h_c = fit(Histogram, res_α_OR7, bins_hist)
h_c = fit(Histogram, res_α_OR7, nbins=nbins_plot)
bc_c = midpoints(h_c.edges[1]) # bin centers
bs_c = fill(h_c.edges[1].step.hi, length(bc_c)) # bin sizes

#h_d = fit(Histogram, res_δ_OR7, bins_hist)
h_d = fit(Histogram, res_δ_OR7, nbins=nbins_plot)
bc_d = midpoints(h_d.edges[1]) # bin centers
bs_d = fill(h_d.edges[1].step.hi, length(bc_d)) # bin sizes
;

In [ ]:
minimum(res_α_OR6), maximum(res_α_OR6)

In [ ]:
minimum(res_δ_OR6), maximum(res_δ_OR6)

In [ ]:
minimum(res_α_OR7), maximum(res_α_OR7)

In [ ]:
minimum(res_δ_OR7), maximum(res_δ_OR7)

In [ ]:
mean(res_α_OR6), mean(res_δ_OR6)

In [ ]:
mean(res_α_OR7), mean(res_δ_OR7)

In [ ]:
# use findall instead of filter, since we're looking for the indices of ZTF observations
#ztf_obs_indx = findall(x->x=="I41", radec_all.obscode)

In [ ]:
#res_α_OR7[ztf_obs_indx]

In [ ]:
#res_δ_OR7[ztf_obs_indx]

In [ ]:
#sqrt(  mean(res_δ_OR7[ztf_obs_indx].^2 .+ res_α_OR7[ztf_obs_indx].^2)  )

In [ ]:
angv = vcat(range(0, length=1000, stop=prevfloat(2pi)), 0)
sigma_factor = 3.0
σ_cos_ang = sigma_factor*cos.(angv)
σ_sin_ang = sigma_factor*sin.(angv)

cov_αδ_cholf_OR6 = cholesky(cov(hcat(res_α_OR6, res_δ_OR6)))
cov_αδ_cholf_OR7 = cholesky(cov(hcat(res_α_OR7, res_δ_OR7)))

ell_3σ_x_OR6 = mean(res_α_OR6) .+ cov_αδ_cholf_OR6.L[1,1].*σ_cos_ang .+ cov_αδ_cholf_OR6.L[1,2].*σ_sin_ang
ell_3σ_y_OR6 = mean(res_δ_OR6) .+ cov_αδ_cholf_OR6.L[2,1].*σ_cos_ang .+ cov_αδ_cholf_OR6.L[2,2].*σ_sin_ang

ell_3σ_x_OR7 = mean(res_α_OR7) .+ cov_αδ_cholf_OR7.L[1,1].*σ_cos_ang .+ cov_αδ_cholf_OR7.L[1,2].*σ_sin_ang
ell_3σ_y_OR7 = mean(res_δ_OR7) .+ cov_αδ_cholf_OR7.L[2,1].*σ_cos_ang .+ cov_αδ_cholf_OR7.L[2,2].*σ_sin_ang
;

In [ ]:
gap = 0.01
xlim_plt = 3.5
ymin_plt = -5.0
ymax_plt = 5.0

Gaston.set(preamble="set grid; set style circle radius 0.02")

p1 = Gaston.plot(
    bc_a,
    h_a.weights./2, 
    supp=[bs_a./2 h_a.weights./2], 
    curveconf="w boxxy notit fs transparent solid 0.4", 
    Axes(
        #linetype = :tab10,
        axesconf="set lmargin at screen 0.1; set rmargin at screen 0.85; set tmargin at screen 0.99; set bmargin at screen 0.8+$gap",
        xrange = (-xlim_plt, xlim_plt),
        format = "''",
        label="1 '(b)' at graph 0.05,0.8"
    ),
    handle = 1
)
Gaston.plot!(
    bc_c,
    h_c.weights./2, 
    supp=[bs_c./2 h_c.weights./2], 
    curveconf="w boxxy notit fs transparent solid 0.4"
)

p2 = Gaston.plot(
    res_α_OR6, 
    res_δ_OR6, 
    curveconf="w circles fill transparent solid 0.4 noborder tit 'gravity-only'",
    Axes(
        #linetype = :tab10,
        key="at screen 0.835, screen 0.78 box spacing 1.2 height 0.4",  #"outside top right box spacing 1.2 height 1",
        axesconf="set lmargin at screen 0.1; set rmargin at screen 0.85; set tmargin at screen 0.8; set bmargin at screen 0.1",
        xrange = (-xlim_plt, xlim_plt),
        yrange = (ymin_plt, ymax_plt),
        format = "\"%g\"",
        xlabel="'\$\\alpha\\cos(\\delta)\$ \$[\$arcsec\$]\$'",
        ylabel="'\$\\delta\$ \$[\$arcsec\$]\$'",
        label="1 '(a)' at graph 0.05,0.925"
    ), 
    handle = 2
)
Gaston.plot!(res_α_OR7, res_δ_OR7, 
    curveconf="w circles fill transparent solid 0.4 noborder tit 'non-grav.'"
)
Gaston.plot!([mean(res_α_OR6)], [mean(res_δ_OR6)], curveconf="w p pt 4 ps 1.5 lw 8") # tit 'mean R.A./Dec (OR6)'
Gaston.plot!([mean(res_α_OR7)], [mean(res_δ_OR7)], curveconf="w p pt 6 ps 1.5 lw 8") # tit 'mean R.A./Dec (OR7)'
Gaston.plot!(ell_3σ_x_OR6, ell_3σ_y_OR6, curveconf="w l lw 8 lc 3")
Gaston.plot!(ell_3σ_x_OR7, ell_3σ_y_OR7, curveconf="w l lw 8 lc 4")

p4 = Gaston.plot(
    h_b.weights./2,
    bc_b,
    supp=[h_b.weights./2 bs_b./2],
    curveconf="w boxxy notit fs transparent solid 0.4",
    Axes(
        #linetype = :tab10,
        axesconf="set lmargin at screen 0.85+$gap; set rmargin at screen 0.99; set tmargin at screen 0.8; set bmargin at screen 0.1",
        yrange = (ymin_plt, ymax_plt),
        format = "''",
        label="1 '(c)' at graph 0.15,0.925"
    ),
    handle = 4
)
Gaston.plot!(
    h_d.weights./2,
    bc_d,
    supp=[h_d.weights./2 bs_d./2],
    curveconf="w boxxy notit fs transparent solid 0.4"
)

Gaston.plot([p1 nothing ; p2 p4])

#Gaston.save(term = "cairolatex", output = "radec_hist_NEW_SANSSERIF.tex", saveopts = "pdf standalone color dashed transparent size 7in,5in font ',12'")
#run(`lualatex radec_hist_NEW_SANSSERIF.tex`)

In [ ]:
?writedlm

In [ ]:
open("optical_residuals_OR6.txt", "w") do io
    writedlm(io, [res_α_OR6 res_δ_OR6])
end

In [ ]:
length(res_α)

In [ ]:
mean(w_α.*res_α(x_OR7)), std(w_α.*res_α(x_OR7))

In [ ]:
mean(w_δ.*res_δ(x_OR7)), std(w_δ.*res_δ(x_OR7))

In [ ]:
mean(res_τ(x_OR7)), std(res_τ(x_OR7))

In [ ]:
mean(res_ν(x_OR7)), std(res_ν(x_OR7))

## OR6

In [ ]:
mean(res_τ(x_OR6)), std(res_τ(x_OR6))

In [ ]:
mean(res_ν(x_OR6)), std(res_ν(x_OR6))

In [ ]:
ind_del_0506 = 1:2
ind_del_1213 = 3:17
ind_del_2121 = 18:20 #18:19

ind_dop_0506 = 1:5
ind_dop_1213 = 6:29
ind_dop_2121 = 30:30
;

In [ ]:
del_index = findall(x->x, deldop_all.delay_index)
dop_index = findall(x->x, deldop_all.doppler_index)
length(del_index), length(dop_index)

In [ ]:
nrow(deldop_all)

In [ ]:
del_dates_plot_ = (deldop_all |> @filter(_.delay_index) |> DataFrame).dt_utc_obs
dop_dates_plot_ = (deldop_all |> @filter(_.doppler_index) |> DataFrame).dt_utc_obs

del_dates_plot = Dates.format.(del_dates_plot_, "dd/mm/yy HHMM")
dop_dates_plot = Dates.format.(dop_dates_plot_, "dd/mm/yy HHMM")

length(del_dates_plot), length(dop_dates_plot)

# Plots: radar astrometry post-fit residuals

In [ ]:
p1 = Gaston.plot(
    del_index,
    res_τ(x_OR6),
    supp = σ_del_all,
    key="box spacing 1.2 height 0.4", #at screen 0.835, screen 0.78 
    curveconf = "u 1:2:3 w errorbars pt 6 lw 4 tit 'gravity-only'",
    Axes(
        #axesconf="set lmargin at screen 0.15; set rmargin at screen 0.95; set bmargin at screen 0.5",
        axesconf="set lmargin at screen 0.11",
        #linetype = :tab10,
        bars = 2,
        ylabel = "'Time-delay resid. \$[\$us\$]\$'",
        xrange = (0,39),
        xtics = "1,2,40",
        format = "y '%1.1f'",
        yrange=(-7,8),
        label="1 '(a)' at graph 0.05,0.9"#,
        #title = "'gravity-only'"
    ),
    handle = 1
)
Gaston.plot!(
    del_index,
    res_τ(x_OR7),
    supp = σ_del_all,
    curveconf = "u 1:2:3 w errorbars pt 4 lw 4 tit 'non-gravitational'",
    lc = 2
)

p2 = Gaston.plot(
    dop_index,
    res_ν(x_OR6),
    supp = σ_dop_all,
    curveconf = "u 1:2:3 w errorbars pt 6 lw 4 tit 'gravity-only'",
    key="box spacing 1.2 height 0.4",
    Axes(
        #axesconf="set lmargin at screen 0.15; set rmargin at screen 0.95; set tmargin at screen 0.5",
        axesconf="set lmargin at screen 0.11",
        #linetype = :tab10,
        bars = 2,
        ylabel = "'Doppler shift resid. \$[\$Hz\$]\$'",
        xrange = (0,39),
        xtics = "1,2,40",
        format = "y '%1.1f'",
        yrange=(-0.6,0.8),
        label="1 '(b)' at graph 0.05,0.9" #,
        # title = "'non-gravitational'"
    ),
    handle = 2
)
Gaston.plot!(
    dop_index,
    res_ν(x_OR7),
    supp = σ_dop_all,
    curveconf = "u 1:2:3 w errorbars pt 4 lw 4 tit 'non-gravitational'",
    lc = 2
)


Gaston.plot([p1; p2])

#Gaston.save(term = "cairolatex", output = "deldop_residuals.tex", saveopts = "pdf standalone color dashed transparent size 10in,7in font ',14'")
#run(`pdflatex deldop_residuals.tex`)

In [ ]:
deldop_dates_plot_xaxis = vcat("01/06/04 0000", del_dates_plot, "31/03/21 2359");

In [ ]:
tmargin_bmargin_gap = 0.03
bmargin_ths_plt = 0.5 + tmargin_bmargin_gap
tmargin_ths_plt = 0.965
axsbrk_v = 1.25*0.01
axsbrk_h = 1.25*0.005
lw_ths_plt = 3
rmargin_ths_plt = 0.3

p1 = Gaston.plot(
    del_dates_plot,
    res_τ(x_OR6),
    supp = σ_del_all,
    curveconf = "u 1:3:4 w errorbars pt 6 lw $lw_ths_plt",
    Axes(
        #linetype = :tab10,
        axesconf = "set rmargin at screen 0.3; set tmargin at screen $tmargin_ths_plt; set bmargin at screen $bmargin_ths_plt",
        xdata   = "time",
        timefmt = "'%d/%m/%y %H%M'",
        xtics = false, #"format '' time (\"01/01/05\",\"01/01/06\")", # '%d/%b/%y'
        xrange = "[\"01/12/04\":\"30/05/06\"]",
        #xlabel  = "\"Date\"",
        bars = false, #2,
        ylabel = "'Time-delay resid. (O-C) [\$\\mu\$s]'",
        border = "1+2+4",
        ytics = ["nomirror", "format '%1.1f'"],
        yrange = "[-8.0:6.0]",
        arrow = [
            "from screen 0.3-$axsbrk_h,$bmargin_ths_plt-$axsbrk_v to screen 0.3+$axsbrk_h,$bmargin_ths_plt+$axsbrk_v nohead",
            "from screen 0.3-$axsbrk_h,$tmargin_ths_plt-$axsbrk_v to screen 0.3+$axsbrk_h,$tmargin_ths_plt+$axsbrk_v nohead"
            ],
        label="1 '(a)' at graph 0.2,0.9",
        grid=false
        #arrow = "from screen 0.3-0.01,1.0-0.01 to screen 0.3+0.01,1.0+0.01 nohead"
        #format = "x ''"
        #title = "'Time-delay resid. (O-C)'"
    ),
    handle = 1
)
Gaston.plot!(
    del_dates_plot,
    res_τ(x_OR7),
    supp = σ_del_all,
    curveconf = "u 1:3:4 w errorbars pt 4 lw $lw_ths_plt",
    lc = 2
)
Gaston.plot!(deldop_dates_plot_xaxis, zeros(length(deldop_dates_plot_xaxis)), curveconf=" u 1:3 w l lc rgb 'gray' lw 4 dt 2")

p2 = Gaston.plot(
    del_dates_plot,
    res_τ(x_OR6),
    supp = σ_del_all,
    curveconf = "u 1:3:4 w errorbars pt 6 lw $lw_ths_plt",
    Axes(
        #linetype = :tab10,
        xdata   = "time",
        timefmt = "'%d/%m/%y %H%M'",
        axesconf = "set lmargin at screen 0.35; set tmargin at screen $tmargin_ths_plt; set rmargin at screen 0.8; set bmargin at screen $bmargin_ths_plt",
        xtics = false, #"format '' time (\"01/01/13\",\"01/02/13\",\"01/03/13\")", # '%d/%b/%y'
        xrange = "[\"15/12/12\":\"20/03/13\"]",
        #xlabel  = "\"Date\"",
        bars = false, #2,
        #tics="scale 0",
        ytics = false, #[false, "(\"\" 0)"], #"nomirror",
        border = "1+4",
        yrange = "[-8.0:6.0]",
        format = ["x ''", "y ''"], #%1.1f
        arrow = [
            "from screen 0.35-$axsbrk_h,$bmargin_ths_plt-$axsbrk_v to screen 0.35+$axsbrk_h,$bmargin_ths_plt+$axsbrk_v nohead",
            "from screen 0.35-$axsbrk_h,$tmargin_ths_plt-$axsbrk_v to screen 0.35+$axsbrk_h,$tmargin_ths_plt+$axsbrk_v nohead",
            "from screen 0.80-$axsbrk_h,$bmargin_ths_plt-$axsbrk_v to screen 0.80+$axsbrk_h,$bmargin_ths_plt+$axsbrk_v nohead",
            "from screen 0.80-$axsbrk_h,$tmargin_ths_plt-$axsbrk_v to screen 0.80+$axsbrk_h,$tmargin_ths_plt+$axsbrk_v nohead"
            ],
        #ylabel = "'[us]'"
    ),
    handle = 2
)
Gaston.plot!(
    del_dates_plot,
    res_τ(x_OR7),
    supp = σ_del_all,
    curveconf = "u 1:3:4 w errorbars pt 4 lw $lw_ths_plt",
    lc = 2
)
Gaston.plot!(deldop_dates_plot_xaxis, zeros(length(deldop_dates_plot_xaxis)), curveconf=" u 1:3 w l lc rgb 'gray' lw 4 dt 2")

p3 = Gaston.plot(
    del_dates_plot,
    res_τ(x_OR6),
    supp = σ_del_all,
    curveconf = "u 1:3:4 w errorbars pt 6 lw $lw_ths_plt",
    Axes(
        #linetype = :tab10,
        xdata   = "time",
        timefmt = "'%d/%m/%y %H%M'",
        axesconf = "set lmargin at screen 0.85; set tmargin at screen 0.965; set bmargin at screen $bmargin_ths_plt",
        xtics = false, #"format '' time (\"03/03/21\",\"11/03/21\")", #'%d/%b/%y'
        xrange = "[\"02/03/21\":\"15/03/21\"]",
        #xlabel = "\"Date\"",
        bars = false, #2,
        #tics="scale 0",
        ytics = false, #[false, "(\"\" 0)"], #"nomirror",
        border = "1+4+8",
        yrange = "[-8.0:6.0]",
        format = ["x ''", "y ''"], #%1.1f
        arrow = [
            "from screen 0.85-$axsbrk_h,$bmargin_ths_plt-$axsbrk_v to screen 0.85+$axsbrk_h,$bmargin_ths_plt+$axsbrk_v nohead",
            "from screen 0.85-$axsbrk_h,$tmargin_ths_plt-$axsbrk_v to screen 0.85+$axsbrk_h,$tmargin_ths_plt+$axsbrk_v nohead"
            ],
        #ylabel = "'[us]'"
    ),
    handle = 3
)
Gaston.plot!(
    del_dates_plot,
    res_τ(x_OR7),
    supp = σ_del_all,
    curveconf = "u 1:3:4 w errorbars pt 4 lw $lw_ths_plt",
    lc = 2
)
Gaston.plot!(deldop_dates_plot_xaxis, zeros(length(deldop_dates_plot_xaxis)), curveconf=" u 1:3 w l lc rgb 'gray' lw 4 dt 2")

Gaston.plot([p1 p2 p3])

#Gaston.save(term = "cairolatex", output = "del_resids.tex", saveopts = "pdf standalone color dashed transparent size 10in,7in font ',14'")
#run(`pdflatex del_resids.tex`)

In [ ]:
@show res_ν(x_OR7)
;

In [ ]:
tmargin_ths_plt = 0.5 - tmargin_bmargin_gap
bmargin_ths_plt = 0.05

Gaston.set(preamble="unset grid")
p4 = Gaston.plot(
    dop_dates_plot,
    res_ν(x_OR6),
    supp = σ_dop_all,
    curveconf = "u 1:3:4 w errorbars pt 6 lw $lw_ths_plt",
    Axes(
        #linetype = :tab10,
        axesconf="set rmargin at screen $rmargin_ths_plt; set tmargin at screen $tmargin_ths_plt; set bmargin at screen $bmargin_ths_plt",
        xdata   = "time",
        timefmt = "'%d/%m/%y %H%M'",
        xtics = "format '%d/%b/%y' time (\"01/01/05\",\"01/01/06\")",
        xrange = "[\"01/12/04\":\"30/05/06\"]",
        #xlabel  = "\"Date\"",
        bars = false,#2,
        ylabel = "'Doppler shift resid. (O-C) [Hz]'",
        border="1+2+4",
        ytics = ["nomirror", "format '%1.1f'"],
        yrange="[-0.5:0.5]",
        arrow = [
            "from screen 0.3-$axsbrk_h,$tmargin_ths_plt-$axsbrk_v to screen 0.3+$axsbrk_h,$tmargin_ths_plt+$axsbrk_v nohead",
            "from screen 0.3-$axsbrk_h,$bmargin_ths_plt-$axsbrk_v to screen 0.3+$axsbrk_h,$bmargin_ths_plt+$axsbrk_v nohead"
            ],
        label="1 '(b)' at graph 0.2,0.9"
        #title = "'Doppler shift resid. (O-C)'"
    ),
    handle = 4
)
Gaston.plot!(
    dop_dates_plot,
    res_ν(x_OR7),
    supp = σ_dop_all,
    curveconf = "u 1:3:4 w errorbars pt 4 lw $lw_ths_plt",
    lc = 2
)
Gaston.plot!(deldop_dates_plot_xaxis, zeros(length(deldop_dates_plot_xaxis)), curveconf=" u 1:3 w l lc rgb 'gray' lw 4 dt 2")

p5 = Gaston.plot(
    dop_dates_plot,
    res_ν(x_OR6),
    supp = σ_dop_all,
    curveconf = "u 1:3:4 w errorbars pt 6 lw $lw_ths_plt",
    Axes(
        #linetype = :tab10,
        xdata   = "time",
        timefmt = "'%d/%m/%y %H%M'",
        axesconf="set lmargin at screen $rmargin_ths_plt+0.05; set rmargin at screen 0.8; set tmargin at screen $tmargin_ths_plt; set bmargin at screen $bmargin_ths_plt",
        xtics="format '%d/%b/%y' time (\"01/01/13\",\"01/02/13\",\"01/03/13\")",
        xrange="[\"15/12/12\":\"20/03/13\"]",
        #xlabel  = "\"Date\"",
        bars = false, #2,
        ytics=false,
        border="1+4",
        yrange="[-0.5:0.5]",
        arrow = [
            "from screen 0.35-$axsbrk_h,$tmargin_ths_plt-$axsbrk_v to screen 0.35+$axsbrk_h,$tmargin_ths_plt+$axsbrk_v nohead",
            "from screen 0.35-$axsbrk_h,$bmargin_ths_plt-$axsbrk_v to screen 0.35+$axsbrk_h,$bmargin_ths_plt+$axsbrk_v nohead",
            "from screen 0.8-$axsbrk_h,$tmargin_ths_plt-$axsbrk_v to screen 0.8+$axsbrk_h,$tmargin_ths_plt+$axsbrk_v nohead",
            "from screen 0.8-$axsbrk_h,$bmargin_ths_plt-$axsbrk_v to screen 0.8+$axsbrk_h,$bmargin_ths_plt+$axsbrk_v nohead"
            ],
        #ylabel = "'[Hz]'"
    ),
    handle = 5
)
Gaston.plot!(
    dop_dates_plot,
    res_ν(x_OR7),
    supp = σ_dop_all,
    curveconf = "u 1:3:4 w errorbars pt 4 lw $lw_ths_plt",
    lc = 2
)
Gaston.plot!(deldop_dates_plot_xaxis, zeros(length(deldop_dates_plot_xaxis)), curveconf=" u 1:3 w l lc rgb 'gray' lw 4 dt 2")

p6 = Gaston.plot(
    dop_dates_plot,
    res_ν(x_OR6),
    supp = σ_dop_all,
    curveconf = "u 1:3:4 w errorbars pt 6 lw $lw_ths_plt",
    Axes(
        #linetype = :tab10,
        xdata   = "time",
        timefmt = "'%d/%m/%y %H%M'",
        axesconf="set lmargin at screen 0.85; set tmargin at screen $tmargin_ths_plt; set bmargin at screen $bmargin_ths_plt",
        xtics="format '%d/%b/%y' time (\"03/03/21\",\"13/03/21\")",
        xrange="[\"02/03/21\":\"15/03/21\"]",
        #xlabel  = "\"Date\"",
        bars = false, #2,
        #format="y ''",
        ytics=false,
        border="1+4+8",
        yrange="[-0.5:0.5]",
        arrow = [
            "from screen 0.85-$axsbrk_h,$tmargin_ths_plt-$axsbrk_v to screen 0.85+$axsbrk_h,$tmargin_ths_plt+$axsbrk_v nohead",
            "from screen 0.85-$axsbrk_h,$bmargin_ths_plt-$axsbrk_v to screen 0.85+$axsbrk_h,$bmargin_ths_plt+$axsbrk_v nohead"
            ],
        #ylabel = "'[Hz]'"
    ),
    handle = 6
)
Gaston.plot!(
    dop_dates_plot,
    res_ν(x_OR7),
    supp = σ_dop_all,
    curveconf = "u 1:3:4 w errorbars pt 4 lw $lw_ths_plt",
    lc = 2
)
Gaston.plot!(deldop_dates_plot_xaxis, zeros(length(deldop_dates_plot_xaxis)), curveconf=" u 1:3 w l lc rgb 'gray' lw 4 dt 2")

Gaston.plot([p4 p5 p6])

#Gaston.save(term = "cairolatex", output = "dop_resids.tex", saveopts = "pdf standalone color dashed transparent size 10in,7in font ',14'")
#run(`pdflatex dop_resids.tex`)

In [ ]:
Gaston.plot([p1 p2 p3; p4 p5 p6])

#Gaston.save(term = "cairolatex", output = "deldop_residuals_dates.tex", saveopts = "pdf standalone color dashed transparent size 10in,7in font ',14'")
#run(`lualatex deldop_residuals_dates.tex`)

# $\Gamma$ matrix eigenvalues/vectors

In [ ]:
Hermitian(Γ_OR7) - Γ_OR7

In [ ]:
norm(Hermitian(Γ_OR7) - Γ_OR7)

In [ ]:
Γ_OR7_cholf = cholesky(Hermitian(Γ_OR7))

In [ ]:
Γ_OR7_cU = Γ_OR7_cholf.U
Γ_OR7_cL = Γ_OR7_cholf.L

In [ ]:
# Compute nominal initial condition in au, au/day, au/day^2
q0_OR7 = [
    -0.18034747703273316,
    0.9406910666200128,
    0.3457360259054398,
    -0.016265942170279046,
    4.392889725556651e-5,
    -0.00039519931615139716,
    0.0, 0.0
] .+ vcat(1e-8ones(6), 1e-14, 1e-13).*x_OR7
@show q0_OR7
;

In [ ]:
# relative scalings analogous to Milani (2005)
scaling_milani_2005 = vcat(  (1/norm(q0_OR7[1:3]))*ones(3), (1/norm(q0_OR7[4:6]))*ones(3), 1/norm(q0_OR7[7]), 1/norm(q0_OR7[8])  )

# jet transport propagation scaling
jt_scaling = vcat(1e-8ones(6), 1e-14, 1e-13)
;

In [ ]:
# compute cov matrix with relative scalings
Γ_OR7_scm05 = diagm(scaling_milani_2005) * ( diagm(jt_scaling)*Γ_OR7*diagm(jt_scaling) ) * diagm(scaling_milani_2005)

In [ ]:
# compute normal matrix with relative scalings
C_OR7_scm05 = inv(Γ_OR7_scm05)

In [ ]:
# compute eigenvalues, eigenvectors of normal matrix with relative scalings
C_eig_OR7_scm05 = eigen(C_OR7_scm05)

In [ ]:
# k₁ * V₁ = 1/√λ₁ * V₁
k1_V1 = C_eig_OR7_scm05.vectors[:,1]/sqrt(C_eig_OR7_scm05.values[1])

In [ ]:
# revert eigenvector of "weak" direction to JT scaling
( k1_V1 ./ scaling_milani_2005 ) ./ jt_scaling

In [ ]:
Taylor1(10) * ( k1_V1 ./ scaling_milani_2005 )

In [ ]:
@show ( ( k1_V1 ./ scaling_milani_2005 )  )  #./ jt_scaling

In [ ]:
σ_OR7

In [ ]:
@show x_OR7 .* jt_scaling

In [ ]:
#@show Γ_OR7